In [122]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import preprocessing
import sklearn.model_selection as ms
import sklearn.metrics as sklm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.random as nr
import seaborn as sns
from datetime import date
from sklearn import linear_model
import scipy.stats as ss
from scipy.stats import skew
import math
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

In [123]:
TrainData = pd.read_csv('train.csv')
TestData=pd.read_csv('test.csv')

In [124]:
for col in ('ExterQual','HeatingQC','FireplaceQu','GarageQual','KitchenQual','BsmtQual','BsmtCond','GarageCond','PoolQC'):
    TrainData[col]=TrainData[col].fillna(0)
    TestData[col]=TestData[col].fillna(0)
    TrainData[col][TrainData[col] == 'Ex'] = 9
    TrainData[col][TrainData[col] == 'Gd'] = 7
    TrainData[col][TrainData[col] == 'TA'] = 5
    TrainData[col][TrainData[col] == 'Fa'] = 3
    TrainData[col][TrainData[col] == 'Po'] = 1
    TrainData[col][TrainData[col] == 'NA'] = 0
    TestData[col][TestData[col] == 'Ex'] = 9
    TestData[col][TestData[col] == 'Gd'] = 7
    TestData[col][TestData[col] == 'TA'] = 5
    TestData[col][TestData[col] == 'Fa'] = 3
    TestData[col][TestData[col] == 'Po'] = 1
    TestData[col][TestData[col] == 'NA'] = 0
    TestData[col].astype('int64')
    TrainData[col].astype('int64')
    

In [125]:
TrainData.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,0,12,2008,WD,Normal,250000


In [126]:
#I want to try to understand the random forest regressor 
#-- features and leaf samples, BEFORE i run this.
# I have 24 Features Chosen

# We adjust with n_estimators and max_features
# max_features=n_features is good for regression problems
#Train = pd.DataFrame(TrainData[['SalePrice','TotRmsAbvGrd','GrLivArea','OverallQual','BldgType','EnclosedPorch','Fireplaces','GarageCars','HouseStyle','LotFrontage','MasVnrType','MiscFeature','MoSold','MSZoning','Neighborhood','OpenPorchSF','PavedDrive','PoolArea','RoofMatl','ScreenPorch','Street','Utilities','WoodDeckSF','YearRemodAdd','YrSold']])
#Test = pd.DataFrame(TestData[['TotRmsAbvGrd','GrLivArea','OverallQual','BldgType','EnclosedPorch','Fireplaces','GarageCars','HouseStyle','LotFrontage','MasVnrType','MiscFeature','MoSold','MSZoning','Neighborhood','OpenPorchSF','PavedDrive','PoolArea','RoofMatl','ScreenPorch','Street','Utilities','WoodDeckSF','YearRemodAdd','YrSold']])
TrainTarget = TrainData[['SalePrice']]
TrainCol = TrainData[['SalePrice','TotRmsAbvGrd','GrLivArea','OverallQual','BldgType','EnclosedPorch','Fireplaces','GarageCars','HouseStyle','LotFrontage','MasVnrType','MiscFeature','MoSold','MSZoning','Neighborhood','OpenPorchSF','PavedDrive','PoolArea','RoofMatl','ScreenPorch','Street','WoodDeckSF','YearRemodAdd','YrSold']]
TestData['SalePrice'] = 0
TestCol = TestData[['SalePrice','TotRmsAbvGrd','GrLivArea','OverallQual','BldgType','EnclosedPorch','Fireplaces','GarageCars','HouseStyle','LotFrontage','MasVnrType','MiscFeature','MoSold','MSZoning','Neighborhood','OpenPorchSF','PavedDrive','PoolArea','RoofMatl','ScreenPorch','Street','WoodDeckSF','YearRemodAdd','YrSold']]

#A hack initially set up for for OneHotEncoder. Complete coverage of categories for dummy variables
TrainTest = TrainCol.append(TestCol) 


In [127]:
TrainTest['MiscFeature']=TrainTest['MiscFeature'].fillna('None')
TrainTest['MasVnrType']=TrainTest['MasVnrType'].fillna('None')
TrainTest['LotFrontage']=TrainTest['LotFrontage'].fillna(0)
TrainTest['GarageCars']=TrainTest['GarageCars'].fillna(0)
TrainTest['MSZoning']=TrainTest['MSZoning'].fillna('None')

TrainTest.to_csv('TrainTestCheck.csv')

In [128]:
TrainTestCol = pd.get_dummies(TrainTest,dummy_na=True)

sc=RobustScaler()
TrainTest=sc.fit_transform(x)

In [129]:
Train = TrainTestCol.iloc[:1460]
Test = TrainTestCol.iloc[1460:]
Train.to_csv('Traincheck.csv')
Test.to_csv('TestCheck.csv')


Train.iloc[:,1:].head(5)
#Test = Test.iloc[:,1:]

,TotRmsAbvGrd,GrLivArea,OverallQual,EnclosedPorch,Fireplaces,GarageCars,LotFrontage,MoSold,OpenPorchSF,PoolArea,...,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofMatl_nan,Street_Grvl,Street_Pave,Street_nan
0,8,1710,7,0,0,2.0,65.0,2,61,0,...,0,0,0,0,0,0,0,0,1,0
1,6,1262,6,0,1,2.0,80.0,5,0,0,...,0,0,0,0,0,0,0,0,1,0
2,6,1786,7,0,1,2.0,68.0,9,42,0,...,0,0,0,0,0,0,0,0,1,0
3,7,1717,7,272,1,3.0,60.0,2,35,0,...,0,0,0,0,0,0,0,0,1,0
4,9,2198,8,0,1,3.0,84.0,12,84,0,...,0,0,0,0,0,0,0,0,1,0


In [130]:
#Setting Up a Poisson Regression Model -- statsmodels.genmod.families.family.Poisson
#Concept - dataset is right skewed. A regression that models a right-skewed distribution is worth investigating.
PoiReg = sm.GLM(Train[['SalePrice']],Train.iloc[:,1:],data=Train, family=sm.families.Poisson()).fit()
#mod1 = smf.glm(formula=formula, data=dta, family=sm.families.Binomial()).fit()


In [131]:
Test.iloc[:,1:].head(5)


,TotRmsAbvGrd,GrLivArea,OverallQual,EnclosedPorch,Fireplaces,GarageCars,LotFrontage,MoSold,OpenPorchSF,PoolArea,...,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofMatl_nan,Street_Grvl,Street_Pave,Street_nan
0,5,896,5,0,0,1.0,80.0,6,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6,1329,6,0,0,1.0,81.0,6,36,0,...,0,0,0,0,0,0,0,0,1,0
2,6,1629,5,0,1,2.0,74.0,3,34,0,...,0,0,0,0,0,0,0,0,1,0
3,7,1604,6,0,1,2.0,78.0,6,36,0,...,0,0,0,0,0,0,0,0,1,0
4,5,1280,8,0,0,2.0,43.0,1,82,0,...,0,0,0,0,0,0,0,0,1,0


In [132]:
Train.iloc[:,1:].head(5)

,TotRmsAbvGrd,GrLivArea,OverallQual,EnclosedPorch,Fireplaces,GarageCars,LotFrontage,MoSold,OpenPorchSF,PoolArea,...,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,RoofMatl_nan,Street_Grvl,Street_Pave,Street_nan
0,8,1710,7,0,0,2.0,65.0,2,61,0,...,0,0,0,0,0,0,0,0,1,0
1,6,1262,6,0,1,2.0,80.0,5,0,0,...,0,0,0,0,0,0,0,0,1,0
2,6,1786,7,0,1,2.0,68.0,9,42,0,...,0,0,0,0,0,0,0,0,1,0
3,7,1717,7,272,1,3.0,60.0,2,35,0,...,0,0,0,0,0,0,0,0,1,0
4,9,2198,8,0,1,3.0,84.0,12,84,0,...,0,0,0,0,0,0,0,0,1,0


In [134]:
#print PoiReg.summary()
Result = pd.DataFrame(PoiReg.predict(Test.iloc[:,1:]))


In [135]:
Result.to_csv('resultreg2.csv')
#Output = pd.DataFrame({'Id':TestData[['Id']], 'SalePrice':Result})
#Output.to_csv('submissiontmb.csv', index=False)